# Import the packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

/home/islam/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
# as_supervised return the data in 2 tuple structure input and target
# with_info provides a table containing info about version ,feature,samples of the dataset 

mnist_dataset , mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

#  Dataset Splitting

In [3]:
mnist_train ,mnist_test=mnist_dataset["train"],mnist_dataset["test"]


# Defining the Validation Sample Size
num_validation_samples=.1*mnist_info.splits["train"].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)

# Defining the Test Sample Size
num_test_samples=mnist_info.splits["test"].num_examples
num_test_samples=tf.cast(num_test_samples,tf.int64)


# Scaling the Images
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255
    return image,label

scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test=mnist_test.map(scale)

# Shuffling the Training and Test Data

In [4]:
BUFFER_SIZE=10000

shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

shuffled_test=scaled_test.shuffle(BUFFER_SIZE)


#  Creating the Validation Data
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)

#  Creating the Training Data

train_data=shuffled_train_and_validation_data.skip(num_validation_samples)




#  Batching the Data:

In [5]:
BATCH_SIZE=100

train_data=train_data.batch(BATCH_SIZE)
validation_data=validation_data.batch(num_validation_samples)


test_data=scaled_test.batch(num_test_samples)



validation_input,validation_target=next(iter(validation_data))

# Model

## Outline the Model

In [62]:
input_size=2
output_size=10
hidden_layer_size=512

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dropout(.3),

    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # Additional hidden layer
        tf.keras.layers.Dropout(.3),

    tf.keras.layers.Dense(output_size,activation='softmax')  # Output layer
])

## Choose the optimizer and loss function

In [63]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])

## Training

In [64]:
EPOCHS_SIZE=10


model.fit(train_data,epochs=EPOCHS_SIZE,validation_data=(validation_input,validation_target),verbose=2)

Epoch 1/10
540/540 - 5s - loss: 0.2734 - accuracy: 0.9176 - val_loss: 0.1226 - val_accuracy: 0.9622
Epoch 2/10
540/540 - 4s - loss: 0.1196 - accuracy: 0.9632 - val_loss: 0.0776 - val_accuracy: 0.9770
Epoch 3/10
540/540 - 4s - loss: 0.0907 - accuracy: 0.9714 - val_loss: 0.0648 - val_accuracy: 0.9788
Epoch 4/10
540/540 - 4s - loss: 0.0746 - accuracy: 0.9765 - val_loss: 0.0519 - val_accuracy: 0.9820
Epoch 5/10
540/540 - 4s - loss: 0.0637 - accuracy: 0.9793 - val_loss: 0.0399 - val_accuracy: 0.9867
Epoch 6/10
540/540 - 4s - loss: 0.0558 - accuracy: 0.9821 - val_loss: 0.0440 - val_accuracy: 0.9848
Epoch 7/10
540/540 - 4s - loss: 0.0487 - accuracy: 0.9840 - val_loss: 0.0341 - val_accuracy: 0.9900
Epoch 8/10
540/540 - 4s - loss: 0.0438 - accuracy: 0.9859 - val_loss: 0.0338 - val_accuracy: 0.9903
Epoch 9/10
540/540 - 4s - loss: 0.0412 - accuracy: 0.9865 - val_loss: 0.0247 - val_accuracy: 0.9930
Epoch 10/10
540/540 - 5s - loss: 0.0361 - accuracy: 0.9880 - val_loss: 0.0263 - val_accuracy: 0.9910

##  Test the model

In [65]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 0s 204ms/step - loss: 0.0669 - accuracy: 0.9825


In [57]:
print("Test loss : {0:.2f}, Test accuracy : {1:.2f}%".format(test_loss,test_accuracy*100))

Test loss : 0.08, Test accuracy : 98.01%


# more complex model

In [119]:
input_size=2
output_size=10
hidden_layer_size=1000

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu'),


    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),  # Additional hidden layer
    tf.keras.layers.Dense(output_size,activation='softmax')  # Output layer
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=["accuracy"])

model.fit(train_data,epochs=10,validation_data=(validation_input,validation_target),verbose=2)

Epoch 1/10
540/540 - 23s - loss: 0.2431 - accuracy: 0.9283 - val_loss: 0.1423 - val_accuracy: 0.9613
Epoch 2/10
540/540 - 23s - loss: 0.1116 - accuracy: 0.9684 - val_loss: 0.0771 - val_accuracy: 0.9788
Epoch 3/10
540/540 - 23s - loss: 0.0785 - accuracy: 0.9775 - val_loss: 0.0767 - val_accuracy: 0.9768
Epoch 4/10
540/540 - 23s - loss: 0.0638 - accuracy: 0.9829 - val_loss: 0.0503 - val_accuracy: 0.9847
Epoch 5/10
540/540 - 23s - loss: 0.0531 - accuracy: 0.9850 - val_loss: 0.0474 - val_accuracy: 0.9855
Epoch 6/10
540/540 - 23s - loss: 0.0500 - accuracy: 0.9861 - val_loss: 0.0821 - val_accuracy: 0.9782
Epoch 7/10
540/540 - 24s - loss: 0.0464 - accuracy: 0.9871 - val_loss: 0.0667 - val_accuracy: 0.9838
Epoch 8/10
540/540 - 25s - loss: 0.0388 - accuracy: 0.9898 - val_loss: 0.0370 - val_accuracy: 0.9908
Epoch 9/10
540/540 - 27s - loss: 0.0360 - accuracy: 0.9904 - val_loss: 0.0382 - val_accuracy: 0.9890
Epoch 10/10
540/540 - 25s - loss: 0.0302 - accuracy: 0.9923 - val_loss: 0.0252 - val_accura

In [126]:
test_loss,test_accuracy=model.evaluate(test_data)

1/1 [==============================] - 1s 619ms/step - loss: 0.0993 - accuracy: 0.9785


In [127]:
print("Test loss : {0:.2f}, Test accuracy : {1:.2f}%".format(test_loss,test_accuracy*100))

Test loss : 0.10, Test accuracy : 97.85%
